## YOLOv8 Tooth Number Detection Training

This notebook trains a YOLOv8 model to detect and classify tooth numbers using the dataset prepared in this project. It:
- installs required dependencies,
- checks your compute device (Apple MPS/CUDA/CPU),
- loads YOLOv8,
- trains on the dataset defined by `data.yaml`, and
- displays training results and basic metrics.


In [16]:
# Install dependencies (ultralytics, opencv-python, matplotlib)
# Note: This cell installs packages into the current Jupyter kernel.
# If you're using a managed kernel, you might not need to run this.
%pip install ultralytics opencv-python matplotlib


Note: you may need to restart the kernel to use updated packages.


In [17]:
# Ensure PyTorch is installed in THIS Jupyter kernel.
# This prints the interpreter path used by the notebook and installs torch/torchvision accordingly.
import sys, subprocess
print("Notebook Python:", sys.version)
print("Interpreter:", sys.executable)

# Install specific versions known to work with Ultralytics used earlier
%pip install --upgrade pip
%pip install torch==2.8.0 torchvision==0.23.0


Notebook Python: 3.12.0 (v3.12.0:0fb18b02c8, Oct  2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)]
Interpreter: /Library/Frameworks/Python.framework/Versions/3.12/bin/python3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Import PyTorch and check available devices.
# On Apple Silicon, MPS (Metal Performance Shaders) accelerates training on the GPU.
# CUDA is for NVIDIA GPUs (not available on macOS/Apple Silicon).
import torch
print("PyTorch version:", torch.__version__)
print("MPS Available:", torch.backends.mps.is_available())
print("CUDA Available:", torch.cuda.is_available())


PyTorch version: 2.8.0
MPS Available: True
CUDA Available: False


In [19]:
# OPTIONAL: Prepare MPS memory to reduce out-of-memory errors on Apple GPU.
# - Clears the MPS cache.
# - Relaxes the MPS high watermark limit (may increase system memory pressure).
import os, torch
try:
    os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'  # disable upper limit; restart kernel if changing this
    if hasattr(torch, 'mps'):
        torch.mps.empty_cache()
        print("Cleared MPS cache and set PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0")
except Exception as e:
    print("MPS prep step skipped:", e)


Cleared MPS cache and set PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0


In [20]:
# Import YOLO from the ultralytics package.
# This provides the high-level `YOLO` class for training, validation, and inference.
from ultralytics import YOLO


In [22]:
# Aggressive low-memory training
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.train(
    data='data.yaml',
    epochs=20,
    imgsz=384,
    batch=4,   # reduce to 2 if still OOM
    device='mps',
    workers=0
)



WARNING ⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8n.pt...


####################################################################      95.1%

Ultralytics 8.3.189 🚀 Python-3.12.0 torch-2.8.0 MPS (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=384, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect

######################################################################## 100.0%


Model summary: 129 layers, 3,017,088 parameters, 3,017,072 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 345.1±161.1 MB/s, size: 90.6 KB)
train: Scanning /Users/revanthguthula/Desktop/yolo-tooth-detection/dataset/labels/train.cache... 397 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 397/397 383584.1it/s 0.0s0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 97, len(boxes) = 11010. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 257.8±38.0 MB/s, size: 55.7 KB)
val: Scanning /Users/revanthguthula/Desktop/yolo-tooth-detection/dataset/labels/val.cache... 100 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 521031.6it/s 0.0s
Plotting labels to runs/detect

RuntimeError: MPS backend out of memory (MPS allocated: 9.07 GiB, other allocations: 8.91 MiB, max allowed: 9.07 GiB). Tried to allocate 2.25 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

### View Training Results

After training, Ultralytics saves outputs under `runs/detect/train/`. The `results.png` plot shows losses and metrics per epoch.


In [ ]:
# Display the training curve image. If the path differs, adjust accordingly.
from IPython.display import Image
Image(filename='runs/detect/train/results.png')


In [ ]:
# Print basic training metrics available on the model object.
# Depending on the Ultralytics version, `model.metrics` may be populated after training.
try:
    metrics = model.metrics
    print(metrics)
except Exception as e:
    print("Metrics not available yet:", e)


### Aggressive low-memory training variant (if OOM persists)
This configuration uses the smallest YOLOv8 model and further reduced settings to minimize memory usage on Apple MPS.


In [ ]:
# Aggressive low-memory training
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.train(
    data='data.yaml',
    epochs=20,
    imgsz=384,
    batch=4,   # reduce to 2 if still OOM
    device='mps',
    workers=0
)
